In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import KFold, train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from lightgbm import LGBMRegressor
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Set seed
SEED = 10
np.random.seed(SEED)

# Load dữ liệu
df = pd.read_csv('C:/Users/Multiplexon/Desktop/data/2/total_selected_augmented.csv')
features = [
    'Transaction Hash_len', 'Original_len', 'signature_len',
    'From_len', 'To_len', 'sender_len', 'paymaster_len',
    'Txn Fee', 'logIndex', 'actualGasCost',
    'actualGasUsed', 'nonce', 'success', 'Blockno', 'DateTime_ts'
]
X = df[features].astype(float)
y = df['Gas Used'].astype(float)

# Chia tập train/validation (không bắt buộc cho K-Fold nhưng vẫn làm cho đồng nhất)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=SEED)

# Scale dữ liệu
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

X_scaled = pd.DataFrame(scaler_X.fit_transform(X), columns=X.columns)
y_scaled = pd.Series(scaler_y.fit_transform(y.values.reshape(-1, 1)).flatten(), name='Gas Used')

# K-Fold
kf = KFold(n_splits=10, shuffle=True, random_state=SEED)

# Đánh giá
mse_scores, mae_scores, rmse_scores, r2_scores = [], [], [], []

for fold, (train_idx, val_idx) in enumerate(kf.split(X_train_scaled), start=1):
    X_tr = X_train_scaled.iloc[train_idx]
    y_tr = y_train_scaled[train_idx]
    X_val = X_train_scaled.iloc[val_idx]
    y_val = y_train_scaled[val_idx]

    model = LGBMRegressor()
             # <- mở nếu LightGBM của bạn hỗ trợ GP )

    # Early stopping dùng callback (tương thích nhiều version LightGBM)
    callbacks = [
        lgb.early_stopping(stopping_rounds=200, verbose=False),
        lgb.log_evaluation(period=0)  # tắt log mỗi vòng
    ]

    model.fit(
        X_tr, y_tr,
        eval_set=[(X_val, y_val)],
        callbacks=callbacks
    )

    # Dự đoán dùng best_iteration_ (nếu có)
    y_pred = model.predict(X_val, num_iteration=getattr(model, "best_iteration_", None))

    # Metrics (trên thang đã scale)
    mse  = mean_squared_error(y_val, y_pred)
    mae  = mean_absolute_error(y_val, y_pred)
    rmse = np.sqrt(mse)
    r2   = r2_score(y_val, y_pred)

    mse_scores.append(mse)
    mae_scores.append(mae)
    rmse_scores.append(rmse)
    r2_scores.append(r2)

  
    print(f"Fold {fold} - MSE: {mse:.6f} | MAE: {mae:.6f} | RMSE: {rmse:.6f} | R²: {r2:.6f}")

# Tổng kết sau tất cả folds
print("\n=== Kết quả trung bình (scaled) ===")
print(f"Avg MSE : {np.mean(mse_scores):.6f}")
print(f"Avg MAE : {np.mean(mae_scores):.6f}")
print(f"Avg RMSE: {np.mean(rmse_scores):.6f}")
print(f"Avg R²  : {np.mean(r2_scores):.6f}")

Fold 1 - MSE: 0.000016 | MAE: 0.000937 | RMSE: 0.004005 | R²: 0.967070
Fold 2 - MSE: 0.000033 | MAE: 0.000967 | RMSE: 0.005737 | R²: 0.936742
Fold 3 - MSE: 0.000018 | MAE: 0.000911 | RMSE: 0.004187 | R²: 0.967241
Fold 4 - MSE: 0.000019 | MAE: 0.001119 | RMSE: 0.004347 | R²: 0.955468
Fold 5 - MSE: 0.000119 | MAE: 0.000980 | RMSE: 0.010904 | R²: 0.815124
Fold 6 - MSE: 0.000027 | MAE: 0.000809 | RMSE: 0.005198 | R²: 0.960109
Fold 7 - MSE: 0.000016 | MAE: 0.001218 | RMSE: 0.004034 | R²: 0.966426
Fold 8 - MSE: 0.000055 | MAE: 0.001396 | RMSE: 0.007396 | R²: 0.902778
Fold 9 - MSE: 0.000032 | MAE: 0.000771 | RMSE: 0.005632 | R²: 0.942693
Fold 10 - MSE: 0.000057 | MAE: 0.001663 | RMSE: 0.007532 | R²: 0.901993

=== Kết quả trung bình (scaled) ===
Avg MSE : 0.000039
Avg MAE : 0.001077
Avg RMSE: 0.005897
Avg R²  : 0.931564
